# Plotting 

**Outline**:

1. Quick overview of plotting with yt
2. Choose your own plotting adventure...

## Plot types in yt

### Spatial Plots: reductions in data spatial dimensionality

![](solutions/figures/intro_figure_00_01_concatenated.png)


2D Spatial plots can be **axis-aligned** or **off-axis** (oriented with an arbitrary plane)

### Spatial Plots for Particles

![](solutions/figures/intro_figure_00_04_concatenated.png)

(SPH off axis support in yt 4.4.0!)

### Plots of statistical distributions

1D & 2D binned statistics with weighting: 

![](solutions/figures/intro_figure_00_02_concatenated.png)


### Volume rendering (more tomorrow...)

![](solutions/figures/intro_figure_00_03_concatenated.png)


## General Usage for Plotting API:

1. generate a plot object with some initital settings
2. optionally use plot **callbacks** to modify that plot
3. call method to display or save 

Two types of callbacks: 

* plot "adjustments" that modify existing plot attributes, method names start with `set_*`
* plot annotations that add new data, method names start with `annotate_*`


In [ ]:
import yt 
ds = yt.load_sample("IsolatedGalaxy")

slc = yt.SlicePlot(ds, 'x', ('gas', 'density'), width=(0.5, 'Mpc')) # create a plot

# some plot adjustments
slc.set_cmap(('gas', 'density'), 'kelp_r')
slc.set_axes_unit('nm')

# some annotations
slc.annotate_grids()
slc.annotate_velocity(normalize=True)

# display the result (don't really have to do this in the notebook cell...)
slc.show()

## Quick examples of plot types

In [ ]:
slc = yt.SlicePlot(ds, 
                   'z', 
                   ('gas', 'density'), 
                   width=(0.5, 'Mpc'))
slc.set_figure_size(3)
slc.annotate_title('axis-aligned SlicePlot')
slc

In [ ]:
slc = yt.SlicePlot(ds, 
                   (1., 1., 1.), 
                   ('gas', 'density'), 
                   width=(0.5, 'Mpc'))
slc.set_figure_size(3)
slc.annotate_title('off-axis SlicePlot')
slc

In [ ]:
prj = yt.ProjectionPlot(ds, 'z', 
                        ('gas', 'density'), 
                        width=(0.5, 'Mpc'))
prj.set_figure_size(3)
prj.annotate_title('axis-aligned ProjectionPlot')

In [ ]:
prj = yt.ProjectionPlot(ds, 
                        (1., 1., 1.), 
                        ('gas', 'density'), 
                        width=(0.5, 'Mpc'))
prj.set_figure_size(3)
prj.annotate_title('off-axis ProjectionPlot')

Same for SPH data!

In [ ]:
ln = yt.LinePlot(ds, 
                 [('gas', 'density'),], 
                 ds.domain_left_edge, 
                 ds.domain_right_edge, 
                 npoints=1000, 
                 figure_size=(3,3))
ln.annotate_title(('gas', 'density'), 'LinePlot')
ln

In [ ]:
pp = yt.ProfilePlot(ds.all_data(), ("gas", "temperature"), ("gas", "density"), weight_field=None)
pp.annotate_title('ProfilePlot')
pp

In [ ]:
pp = yt.PhasePlot(
    ds.all_data(), ("gas", "density"), ("gas", "temperature"), [("gas", "mass")], weight_field=None
)
pp.annotate_title('PhasePlot')
pp

quick aside: 

### **but I just want a histogram!**

First: are you suuuuuure?

In [ ]:
ds = yt.load_sample('enzo_tiny_cosmology')
slc = yt.SlicePlot(ds, 'z', ('gas', 'density'))
slc.annotate_cell_edges()  # a callback annotation!
slc

In [ ]:
p = yt.ProjectionPlot(ds, 'z', ('index', 'ones'), method='sum')
p.set_colorbar_label(('index', 'ones'), 'number of grid cells')
p


**with variable-resolution data, raw histograms will reflect grid structure!**

SO if you're **still** sure you want a raw histogram that just counts occurrences (maybe you have uniformly gridded data, or you're doing some debugging), you can do that with:

In [ ]:
pp = yt.PhasePlot(
    ds.all_data(), 
    ("gas", "density"), 
    ("gas", "temperature"), 
    [("index", "ones")], 
    weight_field=None,     
)
pp

## Some limitations

Not all plot types and annotations support all datasets and geometries:

**non-cartesian** coordinates: 

* For spatial plots, axis-aligned slicing only
* No volume rendering (will come back to this tomorrow)

Datasets that are not grid-based:

* SPH Data: Volume rendering API not supported, good support for slicing and projections though!
* Unstructured grids: Slicing only

Some plot annotations require certain fields (`annotate_magnetic_field`) or data types (`annotate_mesh_lines` for unstructured grids only)